코드 4-1

In [1]:
!pip install ultralytics opencv-python torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.5 MB/s eta 0:00:00


In [2]:
import cv2
from ultralytics import YOLO
from PIL import Image

# 1. 모델 로딩
model = YOLO("yolov8n.pt")

# 2. 이미지 불러오기
img_path = "person.png"
image = cv2.imread(img_path)

# 3. YOLO 탐지 실행
results = model.predict(source=image, conf=0.5, verbose=False)[0]

# 4. 탐지된 객체 중 'person' 클래스만 crop
target_class = "person"
for i, box in enumerate(results.boxes):
    cls_id = int(box.cls[0].item())
    cls_name = model.names[cls_id]

    if cls_name == target_class:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        cropped = image[y1:y2, x1:x2]

        # 5. 잘라낸 이미지 저장
        save_path = f"cropped_{target_class}_{i}.jpg"
        cv2.imwrite(save_path, cropped)
        print(f"📁 저장 완료: {save_path}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
📁 저장 완료: cropped_person_0.jpg
📁 저장 완료: cropped_person_1.jpg
📁 저장 완료: cropped_person_2.jpg
📁 저장 완료: cropped_person_3.jpg


코드 4-2

In [5]:
import cv2
from ultralytics import YOLO

# 1. 모델 로딩 및 이미지 불러오기
model = YOLO("yolov8n.pt")
image = cv2.imread("/content/group.png")
result = model.predict(source=image, conf=0.5, verbose=False)[0]

# 2. 대체 이미지 로드 (예: 이모지, 마스킹 아이콘)
replacement_img = cv2.imread("/content/imoticon.png")  # 반드시 배경 투명 제외하고 RGB 이미지로 준비

# 3. 탐지 결과 후처리
for box in result.boxes:
    cls_id = int(box.cls[0].item())
    cls_name = model.names[cls_id]
    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

    roi = image[y1:y2, x1:x2]

    # 블러 처리
    # blurred = cv2.GaussianBlur(roi, (51, 51), 0)
    # image[y1:y2, x1:x2] = blurred

    # 색상 마스킹 (빨간 박스 채우기)
    # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), -1)

    # 대체 이미지 삽입
    resized_icon = cv2.resize(replacement_img, (x2 - x1, y2 - y1))
    image[y1:y2, x1:x2] = resized_icon

# 4. 결과 저장
cv2.imwrite("output_processed.jpg", image)
print("💾 결과 이미지 저장 완료: output_processed.jpg")


💾 결과 이미지 저장 완료: output_processed.jpg


코드 4-3

In [9]:
!pip install  pyngrok -q

1) 토큰 발급

https://dashboard.ngrok.com/signup
 에서 가입 → 이메일 인증

https://dashboard.ngrok.com/get-started/your-authtoken
 에서 Authtoken 복사

🔹 2. Postman 설치 및 실행

Postman  다운로드 : https://www.postman.com/downloads/

설치 후 실행


In [14]:
NGROK_TOKEN = "30y0W9RbrSxLG0bYesn4GdQu8oW_6AZ5YQTBdmQSRo1Hcijnf"

from pyngrok import ngrok, conf
conf.get_default().auth_token = NGROK_TOKEN

# Flask + ngrok
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

@app.route("/")
def home():
    return "Hello from Flask (Colab + ngrok)!"

@app.route("/echo", methods=["POST"])
def echo():
    return jsonify({"you_sent": request.get_json()})

# 5000 포트로 Flask 실행 & ngrok 터널 오픈
public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)

app.run(port=5000)

🌐 Public URL: NgrokTunnel: "https://27a95881b739.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [12]:
NGROK_TOKEN = "30y0W9RbrSxLG0bYesn4GdQu8oW_6AZ5YQTBdmQSRo1Hcijnf"

from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
conf.get_default().auth_token = NGROK_TOKEN

from ultralytics import YOLO
import cv2
import numpy as np

app = Flask(__name__)
model = YOLO("yolov8n.pt")

@app.route('/detect', methods=['POST'])
def detect_object():
    if 'image' not in request.files:
        return jsonify({'error': 'No image file provided'}), 400

    file = request.files['image']
    img_array = np.frombuffer(file.read(), np.uint8)
    image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    results = model.predict(source=image, conf=0.5, verbose=False)[0]

    detections = []
    for box in results.boxes:
        cls_id = int(box.cls[0])
        cls_name = model.names[cls_id]
        confidence = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        detections.append({
            'class': cls_name,
            'confidence': round(confidence, 2),
            'bbox': [x1, y1, x2, y2]
        })

    return jsonify({'detections': detections})

# Flask 서버를 ngrok으로 외부에 노출
public_url = ngrok.connect(5000)   # 5000번 포트 연결
print("🌐 Public URL:", public_url)

if __name__ == '__main__':
    app.run(debug=True)

🌐 Public URL: NgrokTunnel: "https://aefb3708e3d0.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


코드 4-4

In [ ]:
from flask import Flask, request, send_file
from ultralytics import YOLO
import cv2
import numpy as np
import io

app = Flask(__name__)
model = YOLO("yolov8n.pt")  # 사전학습된 YOLOv8 모델 로드

@app.route('/detect', methods=['POST'])
def detect_yolo():
    if 'image' not in request.files:
        return {'error': 'No image file provided'}, 400

    file = request.files['image']
    image_data = np.frombuffer(file.read(), np.uint8)
    image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)

    # YOLO 탐지 실행
    results = model.predict(source=image, conf=0.5, verbose=False)[0]

    # 탐지된 객체에 박스 그리기
    for box in results.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # 이미지 인코딩 후 반환
    _, buffer = cv2.imencode('.jpg', image)
    return send_file(
        io.BytesIO(buffer.tobytes()),
        mimetype='image/jpeg',
        as_attachment=False,
        download_name='result.jpg'
    )

if __name__ == '__main__':
    app.run(debug=True, port=5000)
